In [1]:
import pandas as pd
import MalekFinance as mf
import numpy as np
import datetime as dt
from pandas.tseries.offsets import MonthEnd
import statsmodels.api as sm
from tqdm.notebook import tqdm
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 14)
pd.set_option('display.float_format', '{:.4f}'.format)

In [2]:
Size = mf.read('WRDS Data','All Exchanges CRSP PEMRMNO Monthly Size DataFrame 63-22',index_col=0,parse_dates=True)
RET = mf.read('WRDS Data','All Exchanges CRSP PEMRMNO Monthly Returns DataFrame 63-22',index_col=0,parse_dates=True)
formation_dates = pd.date_range('1967-12-31','2022-11-30',freq='M')
PastVol = RET.rolling(12,min_periods=10).std()*(12**0.5)

In [3]:
def LowVol_VW(formation):
    Merged = pd.concat([RET.loc[formation + MonthEnd(1)],Size.loc[formation],PastVol.loc[formation]],axis=1).dropna()
    Merged.columns = ['Return','Size','Vol']
    LowVol = Merged[Merged['Vol'] <= Merged['Vol'].quantile(0.2)]
    MidLowVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.2)) & (Merged['Vol'] <= Merged['Vol'].quantile(0.4))]
    MidVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.4)) & (Merged['Vol'] <= Merged['Vol'].quantile(0.6))]
    MidHighVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.6)) & (Merged['Vol'] <= Merged['Vol'].quantile(0.8))]
    HighVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.8)) & (Merged['Vol'] <= Merged['Vol'].quantile(1))]
    for i in LowVol,MidLowVol,MidVol,MidHighVol,HighVol:
        i['Weight'] = i['Size']/sum(i['Size'])
    LowVol_RET = sum(LowVol['Return']*LowVol['Weight'])
    MidLowVol_RET = sum(MidLowVol['Return']*MidLowVol['Weight'])
    MidVol_RET = sum(MidVol['Return']*MidVol['Weight'])
    MidHighVol_RET = sum(MidHighVol['Return']*MidHighVol['Weight'])
    HighVol_RET = sum(HighVol['Return']*HighVol['Weight'])
    list1 = [LowVol_RET,MidLowVol_RET,MidVol_RET,MidHighVol_RET,HighVol_RET]
    Portfolios = pd.DataFrame(list1).transpose()
    Portfolios['DateTime'] = formation + MonthEnd(1)
    Portfolios.set_index(['DateTime'],inplace=True)
    return Portfolios
PortfoliosdfVW12 = pd.DataFrame()
for i in formation_dates:
    PortfoliosdfVW12 = pd.concat([PortfoliosdfVW12,LowVol_VW(i)])
PortfoliosdfVW12.columns = ['Low Vol','Mid Low Vol','Mid Vol','Mid High Vol','High Vol']
PortfoliosdfVW12['L/S'] = mf.long_short(PortfoliosdfVW12)
PortfoliosdfVW12

,Low Vol,Mid Low Vol,Mid Vol,Mid High Vol,High Vol,L/S
DateTime,,,,,,
1968-01-31,-0.0253,-0.0514,-0.0543,-0.0422,-0.0228,-0.0012
1968-02-29,-0.0230,-0.0268,-0.0400,-0.0623,-0.1057,0.0413
1968-03-31,0.0059,0.0054,0.0111,-0.0083,0.0221,-0.0081
1968-04-30,0.0712,0.1012,0.1159,0.1465,0.2157,-0.0722
1968-05-31,0.0205,0.0251,0.0304,0.0637,0.0968,-0.0381
...,...,...,...,...,...,...
2022-08-31,-0.0295,-0.0421,-0.0350,-0.0055,-0.0447,0.0076
2022-09-30,-0.0642,-0.0822,-0.1077,-0.1117,-0.1013,0.0186
2022-10-31,0.0737,0.0829,0.0813,0.0896,-0.0099,0.0418


In [4]:
def LowVol_EW(formation):
    Merged = pd.concat([RET.loc[formation + MonthEnd(1)],Size.loc[formation],PastVol.loc[formation]],axis=1).dropna()
    Merged.columns = ['Return','Size','Vol']
    LowVol = Merged[Merged['Vol'] <= Merged['Vol'].quantile(0.2)]
    MidLowVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.2)) & (Merged['Vol'] <= Merged['Vol'].quantile(0.4))]
    MidVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.4)) & (Merged['Vol'] <= Merged['Vol'].quantile(0.6))]
    MidHighVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.6)) & (Merged['Vol'] <= Merged['Vol'].quantile(0.8))]
    HighVol = Merged[(Merged['Vol'] >= Merged['Vol'].quantile(0.8)) & (Merged['Vol'] <= Merged['Vol'].quantile(1))]
    for i in LowVol,MidLowVol,MidVol,MidHighVol,HighVol:
        i['Weight'] = 1/len(i)
    LowVol_RET = sum(LowVol['Return']*LowVol['Weight'])
    MidLowVol_RET = sum(MidLowVol['Return']*MidLowVol['Weight'])
    MidVol_RET = sum(MidVol['Return']*MidVol['Weight'])
    MidHighVol_RET = sum(MidHighVol['Return']*MidHighVol['Weight'])
    HighVol_RET = sum(HighVol['Return']*HighVol['Weight'])
    list1 = [LowVol_RET,MidLowVol_RET,MidVol_RET,MidHighVol_RET,HighVol_RET]
    Portfolios = pd.DataFrame(list1).transpose()
    Portfolios['DateTime'] = formation + MonthEnd(1)
    Portfolios.set_index(['DateTime'],inplace=True)
    return Portfolios
PortfoliosdfEW12 = pd.DataFrame()
for i in formation_dates:
    PortfoliosdfEW12 = pd.concat([PortfoliosdfEW12,LowVol_EW(i)])
PortfoliosdfEW12.columns = ['Low Vol','Mid Low Vol','Mid Vol','Mid High Vol','High Vol']
PortfoliosdfEW12['L/S'] = mf.long_short(PortfoliosdfEW12)
PortfoliosdfEW12

,Low Vol,Mid Low Vol,Mid Vol,Mid High Vol,High Vol,L/S
DateTime,,,,,,
1968-01-31,0.0214,0.0058,0.0125,0.0160,0.0775,-0.0281
1968-02-29,-0.0170,-0.0331,-0.0536,-0.0685,-0.0986,0.0408
1968-03-31,-0.0084,0.0002,-0.0052,-0.0261,-0.0127,0.0022
1968-04-30,0.0773,0.1179,0.1344,0.1646,0.1991,-0.0609
1968-05-31,0.0440,0.0677,0.0836,0.1058,0.1525,-0.0542
...,...,...,...,...,...,...
2022-08-31,-0.0145,-0.0312,-0.0215,0.0116,0.0194,-0.0170
2022-09-30,-0.0527,-0.0907,-0.1146,-0.1324,-0.1524,0.0499
2022-10-31,0.0293,0.0809,0.0854,0.0575,0.0210,0.0041


In [5]:
# 12 Value Weighted
mf.summary_df(PortfoliosdfVW12)

,Low Vol,Mid Low Vol,Mid Vol,Mid High Vol,High Vol,L/S
Annual Return,11.6900,11.5800,10.8500,9.6300,6.2100,2.7400
Annual Volity,12.7100,16.2900,19.7300,24.5900,31.2900,12.8200
Sharpe Ratio,0.9200,0.7100,0.5500,0.3900,0.2000,0.2100
Max Drawdown,-43.8600,-51.5900,-57.0700,-77.2000,-90.3100,-57.9500
Portfolio Beta,0.7600,1.0300,1.2100,1.4100,1.6200,-0.4300


In [6]:
# 12 Equal Weighted
mf.summary_df(PortfoliosdfEW12)

,Low Vol,Mid Low Vol,Mid Vol,Mid High Vol,High Vol,L/S
Annual Return,12.3200,13.8500,13.6200,13.2000,9.9100,1.2100
Annual Volity,11.4700,16.2400,20.0700,25.0700,31.9000,12.6100
Sharpe Ratio,1.0700,0.8500,0.6800,0.5300,0.3100,0.1000
Max Drawdown,-37.5800,-51.2300,-56.7100,-69.6400,-84.2800,-68.9200
Portfolio Beta,0.6000,0.9000,1.0800,1.2700,1.4300,-0.4100
